In [1]:
import os
import gc
import sys
import glob
import time
import math
import numpy as np
import uproot
import pandas
from functools import partial
from concurrent.futures import ThreadPoolExecutor

import keras
import tensorflow as tf
from keras.utils import plot_model
from keras import regularizers
from keras.models import Sequential, Model, load_model
from keras.layers import Input, Dense, Conv2D, Dropout, AlphaDropout, Activation, BatchNormalization, Flatten
from keras.layers.merge import Concatenate
from keras.callbacks import Callback, ModelCheckpoint, CSVLogger
from keras_tqdm import TQDMNotebookCallback

sys.path.insert(0, "../../python")
from common import *
from DataLoader import DataLoader

C:\Users\konst\AppData\Roaming\Python\Python36\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
class NetSetup:
    def __init__(self, activation, dropout_rate, first_layer_size, last_layer_size, decay_factor, kernel_regularizer):
        self.activation = activation
        if activation == 'relu':
            self.DropoutType = Dropout
            self.kernel_init = 'he_uniform'
            self.apply_batch_norm = True
        elif activation == 'selu':
            self.DropoutType = AlphaDropout
            self.kernel_init = 'lecun_normal'
            self.apply_batch_norm = False
        else:
            raise RuntimeError('Activation "{}" not supported.'.format(activation))
        self.dropout_rate = dropout_rate
        self.first_layer_size = first_layer_size
        self.last_layer_size = last_layer_size
        self.decay_factor = decay_factor
        self.kernel_regularizer = kernel_regularizer
    
def add_block_ending(net_setup, name_format, layer):
    if net_setup.apply_batch_norm:
        norm_layer = BatchNormalization(name=name_format.format('norm'))(layer)
    else:
        norm_layer = layer
    activation_layer = Activation(net_setup.activation, name=name_format.format('activation'))(norm_layer)
    if net_setup.dropout_rate > 0:
        return net_setup.DropoutType(net_setup.dropout_rate, name=name_format.format('dropout'))(activation_layer)
    return activation_layer

def conv_block(prev_layer, filters, kernel_size, net_setup, block_name, n):
    conv = Conv2D(filters, kernel_size, name="{}_conv_{}".format(block_name, n),
                  kernel_initializer=net_setup.kernel_init)(prev_layer)
    return add_block_ending(net_setup, '{}_{{}}_{}'.format(block_name, n), conv)

def reduce_n_features_2d(input_layer, net_setup, block_name):
    conv_kernel=(1, 1)
    prev_layer = input_layer
    current_size = net_setup.first_layer_size
    n = 1
    while True:
        prev_layer = conv_block(prev_layer, current_size, conv_kernel, net_setup, block_name, n)
        if current_size == net_setup.last_layer_size: break
        current_size = max(net_setup.last_layer_size, int(current_size / net_setup.decay_factor))
        n += 1
    return prev_layer

def create_model():
    comp_net_setup = NetSetup('relu', 0.25, 128, 16, 1.4, None)
    #dense_net_setup = NetSetup('relu', 0, 512, 32, 1.4, keras.regularizers.l1(1e-5))
    dense_net_setup = NetSetup('relu', 0.25, 512, 32, 1.4, None)
            
    model_name = "DeepTau2017v2p2"
    input_layer_tau = Input(name="input_tau", shape=(len(input_tau_branches),))
    norm_input_layer_tau = BatchNormalization(name='input_tau_norm')(input_layer_tau)
    input_layers = [ input_layer_tau ]
    
    high_level_features = [ norm_input_layer_tau ]
    for loc in cell_locations:
        reduced_inputs = []
        for comp_id in range(len(component_names)):
            comp_name = component_names[comp_id]
            comp_branches = component_branches[comp_id]
            input_layer_comp = Input(name="input_{}_{}".format(loc, comp_name),
                                     shape=(n_cells_eta[loc], n_cells_phi[loc], len(comp_branches)))
            input_layers.append(input_layer_comp)
            norm_input_layer_comp = BatchNormalization(name='input_{}_{}_norm'.format(loc, comp_name))(input_layer_comp)
            reduced_comp = reduce_n_features_2d(norm_input_layer_comp, comp_net_setup, "{}_{}".format(loc, comp_name))
            reduced_inputs.append(reduced_comp)
        conv_all_start = Concatenate(name="{}_cell_concat".format(loc), axis=3)(reduced_inputs)
        cell_output_size = 32
        prev_layer = conv_block(conv_all_start, cell_output_size, (1, 1), comp_net_setup, "{}_all".format(loc), 1)
        window_size = 4
        current_size = n_cells_eta[loc]
        n = 1
        while current_size > 1:
            win_size = min(current_size, window_size)
            prev_layer = conv_block(prev_layer, cell_output_size, (win_size, win_size), comp_net_setup,
                                    "{}_all_{}x{}".format(loc, win_size, win_size), n)
            n += 1
            current_size -= window_size - 1
            
        cells_flatten = Flatten(name="{}_cells_flatten".format(loc))(prev_layer)
        high_level_features.append(cells_flatten)
        
    prev_layer = Concatenate(name="features_concat")(high_level_features)
    current_size = dense_net_setup.first_layer_size
    n = 1
    while True:
        dense_layer = Dense(current_size, name="final_dense_{}".format(n),
                            kernel_initializer=dense_net_setup.kernel_init,
                            kernel_regularizer=dense_net_setup.kernel_regularizer)(prev_layer)
        prev_layer = add_block_ending(dense_net_setup, 'final_{{}}_{}'.format(n), dense_layer)
        n += 1
        if current_size == dense_net_setup.last_layer_size: break
        current_size = max(dense_net_setup.last_layer_size, int(current_size / dense_net_setup.decay_factor))

    output_layer = Dense(n_outputs, name="final_dense_{}".format(n), kernel_initializer=dense_net_setup.kernel_init)(prev_layer)
    softmax_output = Activation("softmax", name="main_output")(output_layer)

    model = Model(input_layers, softmax_output, name="DeepTau2017v2")
    return model, model_name

In [3]:
def compile_model(model, learning_rate):
    #opt = keras.optimizers.Adam(lr=learning_rate)
    opt = keras.optimizers.Nadam(lr=learning_rate)
    #opt = keras.optimizers.SGD(lr=learning_rate, momentum=0.9, nesterov=True)
    #model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
    metrics = ["accuracy", TauLosses.tau_crossentropy, TauLosses.Le, TauLosses.Lmu, TauLosses.Ljet, TauLosses.sLe, TauLosses.sLmu, TauLosses.sLjet ]
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=metrics, weighted_metrics=metrics)

In [4]:
TauLosses.SetSFs(1, 1, 1)
model, model_name = create_model()
compile_model(model, 1e-3)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_inner_pfCand (InputLayer) (None, 21, 21, 36)   0                                            
__________________________________________________________________________________________________
input_inner_ele (InputLayer)    (None, 21, 21, 39)   0                                            
__________________________________________________________________________________________________
input_inner_muon (InputLayer)   (None, 21, 21, 39)   0                                            
__________________________________________________________________________________________________
input_inner_pfCand_norm (BatchN (None, 21, 21, 36)   144         input_inner_pfCand[0][0]         
__________________________________________________________________________________________________
input_inne

__________________________________________________________________________________________________
input_outer_ele (InputLayer)    (None, 13, 13, 39)   0                                            
__________________________________________________________________________________________________
input_outer_muon (InputLayer)   (None, 13, 13, 39)   0                                            
__________________________________________________________________________________________________
inner_pfCand_dropout_3 (Dropout (None, 21, 21, 65)   0           inner_pfCand_activation_3[0][0]  
__________________________________________________________________________________________________
inner_ele_dropout_3 (Dropout)   (None, 21, 21, 65)   0           inner_ele_activation_3[0][0]     
__________________________________________________________________________________________________
inner_muon_dropout_3 (Dropout)  (None, 21, 21, 65)   0           inner_muon_activation_3[0][0]    
__________

__________________________________________________________________________________________________
outer_pfCand_norm_2 (BatchNorma (None, 13, 13, 91)   364         outer_pfCand_conv_2[0][0]        
__________________________________________________________________________________________________
outer_ele_norm_2 (BatchNormaliz (None, 13, 13, 91)   364         outer_ele_conv_2[0][0]           
__________________________________________________________________________________________________
outer_muon_norm_2 (BatchNormali (None, 13, 13, 91)   364         outer_muon_conv_2[0][0]          
__________________________________________________________________________________________________
inner_pfCand_activation_5 (Acti (None, 21, 21, 32)   0           inner_pfCand_norm_5[0][0]        
__________________________________________________________________________________________________
inner_ele_activation_5 (Activat (None, 21, 21, 32)   0           inner_ele_norm_5[0][0]           
__________

__________________________________________________________________________________________________
inner_muon_conv_7 (Conv2D)      (None, 21, 21, 16)   368         inner_muon_dropout_6[0][0]       
__________________________________________________________________________________________________
outer_pfCand_conv_4 (Conv2D)    (None, 13, 13, 46)   3036        outer_pfCand_dropout_3[0][0]     
__________________________________________________________________________________________________
outer_ele_conv_4 (Conv2D)       (None, 13, 13, 46)   3036        outer_ele_dropout_3[0][0]        
__________________________________________________________________________________________________
outer_muon_conv_4 (Conv2D)      (None, 13, 13, 46)   3036        outer_muon_dropout_3[0][0]       
__________________________________________________________________________________________________
inner_pfCand_norm_7 (BatchNorma (None, 21, 21, 16)   64          inner_pfCand_conv_7[0][0]        
__________

__________________________________________________________________________________________________
outer_ele_conv_6 (Conv2D)       (None, 13, 13, 22)   726         outer_ele_dropout_5[0][0]        
__________________________________________________________________________________________________
outer_muon_conv_6 (Conv2D)      (None, 13, 13, 22)   726         outer_muon_dropout_5[0][0]       
__________________________________________________________________________________________________
inner_all_4x4_conv_1 (Conv2D)   (None, 18, 18, 32)   16416       inner_all_dropout_1[0][0]        
__________________________________________________________________________________________________
outer_pfCand_norm_6 (BatchNorma (None, 13, 13, 22)   88          outer_pfCand_conv_6[0][0]        
__________________________________________________________________________________________________
outer_ele_norm_6 (BatchNormaliz (None, 13, 13, 22)   88          outer_ele_conv_6[0][0]           
__________

__________________________________________________________________________________________________
inner_all_4x4_conv_4 (Conv2D)   (None, 9, 9, 32)     16416       inner_all_4x4_dropout_3[0][0]    
__________________________________________________________________________________________________
outer_all_4x4_conv_1 (Conv2D)   (None, 10, 10, 32)   16416       outer_all_dropout_1[0][0]        
__________________________________________________________________________________________________
inner_all_4x4_norm_4 (BatchNorm (None, 9, 9, 32)     128         inner_all_4x4_conv_4[0][0]       
__________________________________________________________________________________________________
outer_all_4x4_norm_1 (BatchNorm (None, 10, 10, 32)   128         outer_all_4x4_conv_1[0][0]       
__________________________________________________________________________________________________
inner_all_4x4_activation_4 (Act (None, 9, 9, 32)     0           inner_all_4x4_norm_4[0][0]       
__________

__________________________________________________________________________________________________
final_dropout_1 (Dropout)       (None, 512)          0           final_activation_1[0][0]         
__________________________________________________________________________________________________
final_dense_2 (Dense)           (None, 365)          187245      final_dropout_1[0][0]            
__________________________________________________________________________________________________
final_norm_2 (BatchNormalizatio (None, 365)          1460        final_dense_2[0][0]              
__________________________________________________________________________________________________
final_activation_2 (Activation) (None, 365)          0           final_norm_2[0][0]               
__________________________________________________________________________________________________
final_dropout_2 (Dropout)       (None, 365)          0           final_activation_2[0][0]         
__________

In [5]:
def close_file(f_name):
    file_objs = [ obj for obj in gc.get_objects() if ("TextIOWrapper" in str(type(obj))) and (obj.name == f_name)]
    for obj in file_objs:
        obj.close()

In [6]:
class TimeCheckpoint(Callback):
    def __init__(self, time_interval, file_name_prefix):
        self.time_interval = time_interval
        self.file_name_prefix = file_name_prefix
        self.initial_time = time.time()
        self.last_check_time = self.initial_time
    
    def on_batch_end(self, batch, logs=None):
        current_time = time.time()
        delta_t = current_time - self.last_check_time
        if delta_t >= self.time_interval:
            abs_delta_t_h = (current_time - self.initial_time) / 60. / 60.
            self.model.save('{}_b{}_{:.1f}h.h5'.format(self.file_name_prefix, batch, abs_delta_t_h))
            self.last_check_time = current_time

In [7]:
def run_training(train_suffix, model_name, data_loader, epoch, n_epochs):

    train_name = '%s_%s' % (model_name, train_suffix)
    
    cb_acc = []
    for acc_name in ["acc", "weighted_acc"]:
        cb_acc.append(ModelCheckpoint("%s_acc.hdf5" % train_name, monitor="val_%s" % acc_name, save_best_only=True,
                                      save_weights_only=False, mode="max", verbose=1))
    
    cb_losses = []
    for loss_name in ["loss", "tau_crossentropy", "Le", "Lmu", "Ljet",
                      "weighted_tau_crossentropy", "weighted_Le", "weighted_Lmu", "weighted_Ljet"]:
        cb_losses.append(ModelCheckpoint("%s_%s.hdf5" % (train_name, loss_name), monitor="val_%s" % loss_name,
                                         save_best_only=True, save_weights_only=False, mode="min", verbose=1))

    log_name = "%s.log" % train_name
    if os.path.isfile(log_name):
        close_file(log_name)
        os.remove(log_name)
    csv_log = CSVLogger(log_name, append=True)

    pbar = TQDMNotebookCallback(leave_outer=True, show_outer=True, leave_inner = True)
    time_checkpoint = TimeCheckpoint(60*60, '{}_historic'.format(train_name))
    callbacks = [pbar, time_checkpoint, csv_log, *cb_acc, *cb_losses]
    fit_hist = model.fit_generator(data_loader.generator(True), validation_data=data_loader.generator(False),
                                   steps_per_epoch=data_loader.steps_per_epoch, validation_steps=data_loader.validation_steps,
                                   callbacks=callbacks, epochs=n_epochs, initial_epoch=epoch, verbose=0)

    model.save("%s_final.hdf5" % train_name)
    return fit_hist

In [8]:
loader = DataLoader('N:/tau-ml/tuples-v2-t2-hdf/training.h5', 300, 2400, validation_size=7491602,
                    max_queue_size=16, n_passes=-1)

print(loader.file_size, loader.data_size, loader.validation_size)

72491602 65000000 7491602


In [ ]:
fit_hist = run_training('step1', model_name, loader, 0, 4)

In [ ]:
#model.save('p2_l1_try1.h5')